# Домашнее задание по занятию "Рекуррентные сети 3"


1. Возьмите англо-русскую пару фраз (www.manythings.org....org/anki/)
2. Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
3. Попробуйте добавить +1 рекуррентный слой в encoder и decoder
4. Попробуйте заменить GRU ячейки на lstm-ячейки
5. Оцените качество во всех случаях


### импорт библиотек

In [ ]:
%matplotlib inline

from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

###  1.Загрузка и подготовка данных

In [ ]:
# !wget https://download.pytorch.org/tutorial/data.zip
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-09-02 14:56:18--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15824155 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  15.09M  28.0MB/s    in 0.5s    

2023-09-02 14:56:19 (28.0 MB/s) - ‘rus-eng.zip’ saved [15824155/15824155]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:
!mv '/kaggle/working/rus.txt' '/kaggle/working/eng-rus.txt'

In [ ]:
!tail /kaggle/working/eng-rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

#### фукции для подготовки данных

In [ ]:
sep = 'CC-BY'
with open('/kaggle/working/eng-rus.txt') as file:
    for line in file:
        print(line.split(sep, 1)[0])
        break

Go.	Марш!	


In [ ]:
# Создание словаря
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# нормализация текста
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?а-яА-Я]+", r" ", s) # оставляем русс и англ буквы, точку и воскл. знак
    return s

In [ ]:
#  чтение словаря

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    lines = [i.split('\tCC-BY', 1)[0] for i in lines] # отрезаем все, что после \tCC-BY

        # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    # print('input_lang:', input_lang )
    # print('output_lang:', output_lang)
    # print('pairs:', pairs)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 479223 sentence pairs
Trimmed to 27825 sentence pairs
Counting words...
Counted words:
rus 10060
eng 4272
['вы прощены .', 'you re forgiven .']


## 2.Обучение seq2seq по аналогии с занятием

###The Encoder





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# перевод пар в тензоры

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
# функция обучения на одном предложении

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
# функция обучения модели

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    return plot_losses

In [ ]:
# отрисовка графика лоса

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
# фунцкия для проверки качества на сгенерированном предложении

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### Обучение и результат

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

history = trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 56s (- 13m 7s) (5000 6%) 3.1090
1m 47s (- 11m 41s) (10000 13%) 2.6504
2m 39s (- 10m 36s) (15000 20%) 2.3825
3m 31s (- 9m 40s) (20000 26%) 2.1668
4m 23s (- 8m 46s) (25000 33%) 2.0170
5m 14s (- 7m 52s) (30000 40%) 1.8652
6m 7s (- 6m 59s) (35000 46%) 1.7516
6m 59s (- 6m 6s) (40000 53%) 1.6682
7m 50s (- 5m 13s) (45000 60%) 1.5625
8m 43s (- 4m 21s) (50000 66%) 1.4688
9m 36s (- 3m 29s) (55000 73%) 1.4237
10m 28s (- 2m 37s) (60000 80%) 1.3603
11m 21s (- 1m 44s) (65000 86%) 1.2862
12m 14s (- 0m 52s) (70000 93%) 1.2454
13m 6s (- 0m 0s) (75000 100%) 1.1818


In [ ]:
common_history = {}
common_history ['loss'] = {'gru1' : 0, 'gru2' : 0, 'lstm' : 0}
common_history['loss']['gru1']  = round(min(history), 4)

In [ ]:
evaluateRandomly(encoder1, decoder1)

> он дома сеичас .
= he s at home right now .
< he s at home now . <EOS>

> он хитрая лиса .
= he is a sly fox .
< he s a bit fat . <EOS>

> я этим очень горжусь .
= i m very proud of it .
< i m very proud of my . . <EOS>

> ты с друзьями .
= you re with friends .
< you re with friends . <EOS>

> я уверен что том будет пьяныи .
= i m sure tom will be drunk .
< i m sure tom will be late . <EOS>

> он сидит на скамеике .
= he s sitting on the bench .
< he s on his his . <EOS>

> я этим довольна .
= i m happy with that .
< i m happy with that . <EOS>

> мы не глухие .
= we aren t deaf .
< we re not deaf . <EOS>

> мы идем на север .
= we re going north .
< we re going to the . <EOS>

> я съем твои яблоки .
= i m going to eat your apples .
< i m going to eat your . . <EOS>



## 3.Добавление еще +1 рекурентного слоя

###The Encoder





In [ ]:
class EncoderRNN2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru1(output, hidden)
        output, hidden = self.gru2(output, hidden)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder




In [ ]:
class DecoderRNN2(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru1(output, hidden)
        output, hidden = self.gru2(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN2(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN2(hidden_size, output_lang.n_words).to(device)

history = trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 20s (- 18m 45s) (5000 6%) 3.3222
2m 38s (- 17m 9s) (10000 13%) 2.9352
3m 57s (- 15m 49s) (15000 20%) 2.7333
5m 16s (- 14m 31s) (20000 26%) 2.6432
6m 36s (- 13m 12s) (25000 33%) 2.5248
7m 56s (- 11m 54s) (30000 40%) 2.4399
9m 16s (- 10m 35s) (35000 46%) 2.3037
10m 36s (- 9m 17s) (40000 53%) 2.1720
11m 56s (- 7m 57s) (45000 60%) 2.0961
13m 17s (- 6m 38s) (50000 66%) 1.9707
14m 36s (- 5m 18s) (55000 73%) 1.8957
15m 57s (- 3m 59s) (60000 80%) 1.8225
17m 18s (- 2m 39s) (65000 86%) 1.7233
18m 38s (- 1m 19s) (70000 93%) 1.6561
19m 58s (- 0m 0s) (75000 100%) 1.5926


In [ ]:
common_history['loss']['gru2']  = round(min(history), 4)

In [ ]:
evaluateRandomly(encoder1, decoder1)

> я горжусь своеи дочерью .
= i m proud of my daughter .
< i m proud of my cat . <EOS>

> я рада что не сделала этого .
= i m glad i didn t do that .
< i m glad i didn t do that . <EOS>

> я уверен что том будет здесь к .
= i m sure tom will be here by .
< i m sure tom will be here . <EOS>

> сеичас они в одиночестве .
= they re now alone .
< they re now alone now . <EOS>

> надеюсь том скоро позвонит .
= i m hoping tom will call soon .
< i m hoping tom tom tom tom . <EOS>

> я не собираюсь этого делать .
= i m not going to do this .
< i m not going to do that yet . <EOS>

> ты невероятно тупои .
= you re incredibly stupid .
< you re incredibly stupid . <EOS>

> я не закончил с тобои разговаривать .
= i m not finished talking to you .
< i m not finished talking to you . <EOS>

> теперь я женат .
= i m married now .
< i m married now . <EOS>

> наш разговор окончен .
= we re done talking .
< we re out of time . <EOS>



## 4.Замена GRU на LSTM

### The Encoder


In [ ]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, hidden_size)
        # self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder




In [ ]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden  = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

history = trainIters(encoder1, decoder1, 75000, print_every=5000)

29m 54s (- 418m 47s) (5000 6%) 3.8168
61m 0s (- 396m 32s) (10000 13%) 3.5143
91m 59s (- 367m 59s) (15000 20%) 3.4560
123m 6s (- 338m 32s) (20000 26%) 3.4669
153m 58s (- 307m 57s) (25000 33%) 3.4285
184m 58s (- 277m 27s) (30000 40%) 3.4389
216m 2s (- 246m 54s) (35000 46%) 3.4211
246m 48s (- 215m 57s) (40000 53%) 3.3804
277m 32s (- 185m 1s) (45000 60%) 3.4153
308m 9s (- 154m 4s) (50000 66%) 3.3843
338m 51s (- 123m 13s) (55000 73%) 3.3715
369m 21s (- 92m 20s) (60000 80%) 3.3579
400m 8s (- 61m 33s) (65000 86%) 3.3906
430m 41s (- 30m 45s) (70000 93%) 3.3613
461m 9s (- 0m 0s) (75000 100%) 3.3639


In [ ]:
common_history['loss']['lstm']  = round(min(history), 4)

In [ ]:
evaluateRandomly(encoder1, decoder1)

> я слесарь .
= i m a locksmith .
< i m not . . <EOS>

> я поздно спохватился .
= i m too late .
< i m not . . <EOS>

> я не обижена .
= i m not offended .
< i m not . . <EOS>

> вы забавная .
= you re funny .
< i m not . . <EOS>

> вы незаменимы .
= you aren t replaceable .
< i m not . . <EOS>

> здесь ты в безопасности .
= you re safe here .
< i m not . . <EOS>

> я боюсь волков .
= i m afraid of wolves .
< i m not . . <EOS>

> ты настоящии друг .
= you re a real friend .
< i m not . . <EOS>

> я так пить хочу .
= i m so thirsty .
< i m not . . <EOS>

> я это есть не буду .
= i m not eating this .
< i m not . . <EOS>



## Результат сравнения

In [ ]:
compare_result = pd.DataFrame.from_dict(common_history)
compare_result

,loss
gru1,0.9617
gru2,1.3585
lstm,3.0657


Вывод: на данном датасете лучший результат получился у модели с одним слоем GRU.